# COLLABORATIVE FILTERING
 
**Suppose for user A and user B, specified rating are correlated (which can be identfied by algorithm) then it is very likely that the ratings in which only one of the them is specified, are also likely to similar. This similarity can be used to make inferences about incompletely specified value**. This is the main idea of collaborative filtering. There are two types of methods that are commonly used in collaborative filtering, which are referred to as memory based methods and model based methods.

Collaborative filtering models use the collaborative power of ratings given by multiple users to make recommendations.

Let $m$ represents number of users and $n$ represents number of items. Most users have rated only a fraction of items. As a result most of the ratings will be not specified. Let $r_{ij}$ represent the rating of user $i$ and item $j$. Let R represent the rating matrix with $r_{ij}$ as elements. Point to note is **rating matrix R is incomplete**. An example of ratings matrix given in figure.

![](https://drive.google.com/uc?export=view&id=1YNzJz5zSD1YiceAOMtPekx7jUUzBDEGA)


## MEMORY BASED COLLABORATIVE FILTERING

In this ratings on user-item combinations are predicted on basis of their neighborhoods. Because of different type of neighborhoods we have two types here

### 1) User based
In this case the ratings provided by similar users to a target user A are used to make recommendations for A. The predicted ratings for A are computed as **weighted average** of the group that are most similar to A. 

Let $I_{u}$ denote the set of item indices denoted by user $u$. One way to measure similarity between two users is to use **Pearson correlation** between the ratings given by them. We can also other similarity functions like cosine similarity but we will stick to this one here. The pearson correlation between two users $u$ and $v$ is given by 

$$
sim(u,v) =  \frac{\sum_{k\in I_{u} \cap I{v}}(r_{uk}-\mu_{u})(r_{vk}-\mu_{v})}{\sqrt{\sum_{k\in I_{u} \cap I{v}}(r_{uk}-\mu_{u})^{2}}\sqrt{\sum_{k\in I_{u} \cap I{v}}(r_{vk}-\mu_{v})^{2}}}
$$

Here $\mu_{u}$ and $\mu_{v}$ are the respective avergaes of the ratings for user $u$ and $v$. Note the average here is computed only over the items that are both common to $u$ and $v$. 

For example, user $u$ has ratings $[1,3,4,7,6]$ on items and user $v$ has ratings $[2,1,3,?,?]$, then pearson correlation betweem them is 

$$
sim = \frac{(1-2.66)(2-2)+(3-2.66)(1-2)+(4-2.66)*(3-2)}{\sqrt{(1-2.66)^2+(3-2.66)^2+(4-2.66)^2}\sqrt{(2-2)^2+(1-2)^2+(3-2)^2}}
$$

After finding the similiarity between target user and all the users, now can on select top-k users based on similarity. The weighted average of these ratings can be returned as the predicted rating for that item with weights being equal to similarities. Before this we preprocess the rating matrix by **mean centering in row wise fashion**. We can write 
$$
s_{uj} = r_{uj} - \mu_{u}
$$

Now prediction will be given by
$$
\hat{r_{ij}} = \mu_{u} + \frac{\sum_{u\in P_{u}(j)}sim(u,v)s_{u,j}}{\sum_{u\in P_{u}(j)}\lvert sim(u,v) \rvert}
$$

Here $P_{u}(j)$ is the top-k neighbour of user $u$ for item $j$.


The psuedo code for the algorithm will be
```
Compute the similarity between users (cosine or pearson)
For a given user select top-k neighbours. (by similarity)
Rating of user i on item j is predicted as weighted average of the top-k neighbours.
```

### 2) Item Based
In order to make recommendations for target item B, we will deterime the set S of items that are most similar to B. Then in order to predict the rating of any particular user A for item B, the ratings in set S, which are specified by A. The weighted average of these ratings are used as prediction. 

The important distinction is that in item based we will find the **similarties between items** rather than **similarites between users**. The predicted rating will be
$$
\hat{r_{ut}} = \frac{\sum_{j \in Q_{t}(u)}sim(j,t)r_{uj}}{\sum_{j \in Q_{t}(u)}\lvert sim(j,t) \rvert}
$$

The psuedo code for the algorithm will be 
```
Compute the similarity between each item pair.
For a given item, select the top-k items
Rating for user i and item j, is weighted average of ratings given by i in the neighbours.
```

##### Comparision between user based and item based
- Item-based methods often provide more relevant recommendations because of the fact that a user’s own ratings are used to perform the recommendation. As in item-based methods, similar items are identified to a target item, and the user’s own ratings on those items are used to extrapolate the ratings of the target user.

- We can give explanations or interpret item based recommendations. On the other hand, these explanations are harder to address withuser-based methods, because the peer group is simply a set of anonymous users and not directly usable in the recommendation process.

- Finally, item-based methods are more stable with changes to the ratings.

#### STRENGTHS AND WEAKNESS OF MEMORY BASED MODELS

- Because of the simple and intuitive approach of these methods, they are easy to implement and debug.Because of the simple and intuitive approach of these methods, they are easy to implement and debug.

- Main disadvantage of these methods is sparsity. As sparse ratings matrices are more common, finding neighbours will be tough and this will make the prediction not very accurate.Sparsity also creates challenges for robust similarity computation when the number of mutually rated items between two users is small.



## MODEL BASED COLLABORATIVE FILTERING

In model-based methods, a summarized model of the data is created up front, as with supervised or unsupervised machine learning methods. Therefore, the training (or model-building phase) is clearly separated from the prediction phase. Even though neighborhood-based methods were among the earliest collaborative filtering methods and were also among the most popular because of their simplicity, they are not necessarily the most accurate models available today. In fact, some of the most accurate methods are based on model-based techniques in general, and on latent factor models in particular.

### LATENT FACTOR MODELS

In this method, we use dimensionality reduction method to directly estimate the matrix in one shot. The basic idea is to exploit the fact that significant portions of rows and columns of data matrices are highly correlated. As a result, the data has built-in redundancies and the resulting data matrix is often approximated quite well by **low rank matrix**.  

Latent factor models are considered to be state of the art in recommender systems. These models leverage well known dimensionality reduction methods to fill the missing entries.

Factorization is a more general way of approximating a matrix when it is prone to dimensionality reduction because of correlations between columns (or rows). Most dimensionality reduction methods can also be expressed. Matrix factorization methods provide a neat way to leverage all row and column correlations in one shot to estimate the entire data matrix. This sophistication of the approach is one of the reasons that latent factor models have become the state-of-the-art in collaborative filtering.


The key idea is that **any $m \times n$ matrix R of rank $k << \, min(m,n)$ can always be expressed as following product.**

$$
R = UV^{T}
$$

Here U is $m \times k$ matrix and V is $n \times k$ matrix. Even when the matrix R has rank larger than k, it can often be approximately expressed as the product of rank-k factors
$$
R \approx UV^{T}
$$

let us consider the straightforward case in which R is fully observed. If the matrix R is fully specified, then are there are lot of methods to find U and V. But what happens **if we missing entries in R ?**. The main idea here is to **minimize the errors between only observed entries given by R and product $UV^{T}$**. So we have written this as an optimization problem. Once solved , that is if we can find U and V that minimize the errors, **we can estimate the matrix as $UV^{T}$ in one shot**.

One can formulate the optimization problem with respect to matrices U and V in order to acheive this goal 
$$
\min_{U,V} J = \frac{1}{2}\displaystyle\sum_{(i,j)\in S}(r_{i,j}-\displaystyle\sum_{s=1}^{k} u_{is}v_{js})^{2}
$$

where S denotes the set of $(i,j)$ for which $r_{ij}$ is observed. A varitey of **gradient methods** can be used to provide an optimal solution to this factorization.


#### REGULARIZATION

One of the main problems with this approach arises when the ratings matrix R is sparse and relatively few entries are observed. In such cases when observed set is small, overfitting will arise. A common apporach to address overfitting is to use ***regularization***. Here we $l_{2}$ regularization and the main idea is to add square of parameters to the objective.

In case of regularization, our objective will become
$$
\min_{U,V} J = \frac{1}{2}\displaystyle\sum_{(i,j)\in S}(r_{i,j}-\displaystyle\sum_{s=1}^{k} u_{is}v_{js})^{2} + \frac{\lambda}{2}\displaystyle\sum_{i=1}^{m}\displaystyle\sum_{s=1}^{k}u_{is}^{2} + \frac{\lambda}{2}\displaystyle\sum_{j=1}^{n}\displaystyle\sum_{s=1}^{k}v_{js}^{2}
$$

The basic idea is to create a bias in favor of simpler solutions by penalizing large coefficients. This is a standard approach, which is used in many forms of classification and regression, and also leveraged by collaborative filtering. The parameter $\lambda$ is always non-negative and it controls the weight of the regularization
term.

Value of $\lambda$ can be found by using a hold out set and finding the $\lambda$ which will have minimum error on hold out set.

#### STOCHASTIC GRADIENT DESCENT

We can find the gradient of the above function and can use gradient descent to optimize. But in practice, faster convergence is chieved by the stochastic gradient descent method as compared to the batch method.

The algorithm is given below

```
Rating Matrix R
begin
    Randomly initialize U and V;
    while not(covergence) do
    begin
        randomly shuffle entries in S;
        for each (i,j) in s
        begin
            error = r - u.v
            find gradient
            u = u - lr*gradu
            v = v - lr*gradv
        end
        check convergence condition
    end
end
```

This algorithm is same as we do stochastic gradient descent in neural networks. Point to note, final solution will depend on initialization and learning rate of the algorithm.


#### ALTERNATING LEAST SQUARES (ALS)
The stochastic gradient method is an efficient methodology for optimization. On the other hand, it is rather sensitive, both to the initialization and the way in which the step sizes are chosen. Other methods for optimization include the use of alternating least squares(ALS) which is generally more stable.

The basic idea of the apporach is
1. Keeping U fixed, we solve for each of $n$ rows of V by treating the problem as least square regression problem. As we have to solve $n$ least square problem and each problem is independent of other, we can parallelize this step easily.
2. Keeping V fixed, we solve for each of $m$ rows of U by treating the problem as least square regerssion problem. As we have to solve $m$ least square problem and each problem is independent of other, we can parallelize this step as in previous case.


These two steps are iterated to convergence which typically occurs within a small (< 20) number of iterations even for very large matrices consisting of tens of millions of rows or columns. We can also use regularization here as it will be equivalent solving linear regression problem with regularization.

##### WEIGHTED ALERNATING LEAST SQUARES (WALS)

Impilicit matrices meaning the matrix is filled with implicit rating. In this cases, customer
preferences are derived from their activities rather than their explicitly specified ratings. For example, the buying behavior of a customer is implicit. Similarly, many social networks, such as Facebook, use “like” buttons, which provide the
ability to express liking for an item. 

This method is usually **used for implicit rating matrices** in which the matrix is assumed to be fully specified with many zero values. When lot of entries are zero some tricks can be used to make weighted ALS efficient.


It introduces the weights for zero(unobserved) entries, and observed entries. 
The objective will be

$$
\min_{U,V} J = \frac{1}{2}\displaystyle\sum_{(i,j)\in S}w(r_{i,j}-\displaystyle\sum_{s=1}^{k} u_{is}v_{js})^{2}
$$
here
- $w_{is} = w_{0}$   for unobserved entries
- $w_{is} = w_{0} + f(c_{i})$ for observed entries, $c_{i}$ is the number of non zero entries in row $i$.

This type of weighting allows for a more flexible model of the user's preferences and produces better empirical results than the unweighted version. The function $f$ varies according to the dataset and whether ratings are explicit or implicit.


If an item is not seen during training, the system can't create an embedding for it and can't query the model with this item. This issue is often called the **cold-start problem**. WALS handles cold start problem. Given a new item $i_{0}$ not seen in training, if the system has a few interactions with users, then the system can easily compute an embedding  for this item without having to retrain the whole model.